In [1]:
from datetime import datetime, timedelta, timezone

from flask import Flask, flash, render_template, redirect, request, url_for

from flask_login import LoginManager, login_user, logout_user, login_required, current_user

from models import User, Group, Post, db


app = Flask(__name__)
app.config.from_object('config')  # Load configuration from config.py

login_manager = LoginManager(app)
login_manager.login_view = "login_page"

with app.app_context():
    db.init_app(app)
    db.create_all()


In [2]:
import atexit

with app.app_context():
    db.engine.dispose()

# Dispose of the engine on app shutdown
atexit.register(lambda: db.engine.dispose())

<function __main__.<lambda>()>

In [3]:
# Create our first user
with app.test_request_context():
    user = User(username="Emily", is_organizer=True)
    db.session.add(user)
    db.session.add(User(username="Robert", is_organizer=True))
    db.session.add(User(username="Occie", is_organizer=False))
    db.session.commit()

In [4]:
with app.test_request_context():
    print(User.query.all())

[User Emily, is_organizer=True>, User Robert, is_organizer=True>, User Occie, is_organizer=False>]


In [5]:
# Let's create our first group
with app.test_request_context():
    user = User.query.filter_by(username='Emily').first()
    print(user)
    group = Group(group_name='Bakery',description='homemade bakery',
                         create_user=user)
    db.session.add(group)
    db.session.commit()

User Emily, is_organizer=True>


In [6]:
# Let's create our second group
with app.test_request_context():
    user = User.query.filter_by(username='Robert').first()
    print(user)
    db.session.add(Group(group_name='Dundrum PYE',description='Local doggy friendly bar',
                         create_user=user))
    db.session.commit()

User Robert, is_organizer=True>


In [7]:
with app.test_request_context():
    user = User.query.filter_by(username='Robert').first()
    print(user)

User Robert, is_organizer=True>


In [8]:
with app.test_request_context():
    print(Group.query.all())

["Group <group_name=Bakery, is_active= True, create_at (2024-11-07>), "Group <group_name=Dundrum PYE, is_active= True, create_at (2024-11-07>)]


In [9]:
with app.test_request_context():
    user = User.query.filter_by(username='Emily').first()
    group = Group.query.filter_by(create_user=user).first()
    due_date = datetime.now(timezone.utc) + timedelta(days=7)
    db.session.add(Post(title='Chocolate roll cake', context='Lets bake', date_to=due_date,
                        group=group, editor=user ))
    db.session.commit()

In [10]:
with app.test_request_context():
    user = User.query.filter_by(username='Robert').first()
    pye = Group.query.filter_by(create_user=user).first()
    print(user)
    print(pye)
    due_date = datetime.now(timezone.utc) + timedelta(days=10)
    db.session.add(Post(title='Occie party', context='1 year old party', date_to=due_date,
                        group=pye, editor=user ))
    db.session.commit()

User Robert, is_organizer=True>
"Group <group_name=Dundrum PYE, is_active= True, create_at (2024-11-07>)


In [11]:
with app.test_request_context():
    user = User.query.filter_by(username='Emily').first()
    pye = Group.query.filter_by(group_name='Dundrum PYE').first()
    print(user)
    print(pye)
    due_date = datetime.now(timezone.utc) + timedelta(days=10)
    db.session.add(Post(title='Doggy', context='Local doggy party', date_to=due_date,
                        group=pye, editor=user ))
    db.session.commit()

User Emily, is_organizer=True>
"Group <group_name=Dundrum PYE, is_active= True, create_at (2024-11-07>)


In [12]:
with app.test_request_context():
    user = User.query.filter_by(username='Robert').first()
    pye = Group.query.filter_by(create_user=user).first()
    print(pye.posts)

["Post <title=Occie party" created_at=2024-11-07>), "Post <title=Doggy" created_at=2024-11-07>)]


In [13]:
with app.test_request_context():
    patron = User.query.filter_by(user_id=1).first()
    print(f"The user {patron.username} has post:")
    for post in patron.posts:
        print(f"- {post.group.group_name}-{post.post_id} ({post.title}) due by {post.date_fm}")

The user Emily has post:
- Bakery-1 (Chocolate roll cake) due by 2024-11-07 20:45:59.628423
- Dundrum PYE-3 (Doggy) due by 2024-11-07 20:45:59.690754


In [14]:
with app.test_request_context():
    patron = User.query.filter_by(user_id=2).first()
    print(f"The user {patron.username} has post:")
    for post in patron.posts:
        print(f"- {post.group.group_name}-{post.post_id} ({post.title}) due by {post.date_fm}")

The user Robert has post:
- Dundrum PYE-2 (Occie party) due by 2024-11-07 20:45:59.656280


In [15]:
with app.test_request_context():
    group = Group.query.filter_by(group_id=2).first()
    print(f"The group {group.group_name} has post:")
    for post in group.posts:
        print(f"- {post.editor.username} {post.post_id} ({post.title}) due by {post.date_fm}")

The group Dundrum PYE has post:
- Robert 2 (Occie party) due by 2024-11-07 20:45:59.656280
- Emily 3 (Doggy) due by 2024-11-07 20:45:59.690754
